In [53]:
import preprocess.prepare as prepare
import marko
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
data = prepare.CratesData()

Using cached:  crates-io.tar.gz
Cleaning up old crates-io data
Extracting crates-io data


Merging crates-categories relationship: 100%|██████████| 59887/59887 [00:00<00:00, 1098409.50it/s]


In [49]:
crate_example = data['syn']
crate_example_categories_indices = crate_example.category_indices
print(crate_example_categories_indices)

[11, 14]


In [50]:
print(f"Categories for crate {crate_example.name} are [{', '.join([data.categories[i].name for i in crate_example_categories_indices])}]")

Categories for crate syn are [Parser implementations, Development tools::Procedural macro helpers]


In [51]:
print(crate_example.description)

Parser for Rust source code


In [71]:
elem = marko.parse(crate_example.readme).children[6].children[0].children

In [73]:
elem

'Currently this library is geared toward use in Rust procedural macros, but'